# IE 306.02 Assignment 1
In this assignment we have implemented a call center simulation. Clients place a call to front desk operator who then further forwards them to an expert operator. Clients may renage after front desk operator while waiting for expert, expert takes a break sometimes and both queues are FCFS. Exact details of both queues can be found in assignment description.

We have designed the system as two M/M/1 queues put one after another. In the first queue, server is front desk opetor who we modeled as a resource in the SimPy environment. Similarly, expert is the server in the second queue who is also modeled as a resource in SimPy. 

Below, we first describe our code and later interpret the statistics and results we have gathered from the simulation.

First we import the basic libraries.

In [1]:
import simpy
import random
import numpy.random
import numpy as np
from math import log,sqrt
import itertools # for easy list operations
from collections import namedtuple

A little helper function for formatting seconds to desired time format.

In [2]:
def format_time(total_minutes):
    secs = int((total_minutes % 1)*60)
    mins = int(total_minutes % 60)
    hours = int(total_minutes //60) % 24
    days = int(total_minutes//60//24)
    return f" {days:02}:{hours:02}:{mins:02}:{secs:02}"
        
# an object to hold expert queue statistics
Expert_Q_Stat = namedtuple('Expert_Q_Stat', ['enter_e_q', 'exit_e_q'])

Below, we first describe our code and then interpret results.

We ollect and report statistics on: 

* Utilization of the front-desk operator

* Utilization of the expert operators, 

* Average Total Waiting Time,

* `Maximum Total Waiting Time` to `Total System Time Ratio`,

* Average number of people waiting to be served by the expert operator.

Global constants that will be used in the code.

In [3]:
# interarrival rates are exponentially distributed with mean 14.3 min
INTERARRIVAL_MEAN = 14.3 
# service time of first operator is LogNormal distributed with mean 7.2 and sd 2.7
m = 7.2
v = 2.7

FIRST_SERVICE_MEAN = log((m**2)/sqrt(v+m**2))
FIRST_SERVICE_STD = sqrt(log(v/(m**2)+1))


# customers leave queue if not serviced in exponentially distributed time with mean 60 min
RENEGE_TIME_MEAN = 60
# service time of expert is exponentially distributed with mean 10.2
EXPERT_SERVICE_MEAN = 10.2
# number of breaks taken by expert is Poisson distributed with mean 8 breaks
BREAKS_MEAN = 8
BREAK_DUR = 3

# number of customers simulation will be run for
CALLNUM1=1000
CALLNUM2=5000

LAST_EXIT = 0

LAST_CUSTOMER = 1

CUSTOMER_NUM=CALLNUM1


#RANDOM_SEED = 971
#random.seed(RANDOM_SEED)

Here, we define the necessary set of arrays for bookkeeping

In [4]:
# service times for front-desk operator
first_service_times = [] 
# service times for expert 
second_service_times = []
# time spent by a customer while it waits for the operator (Queue waiting time Wq)
first_queue_w_times = [] 
second_queue_w_times = [] 


expert_queue_log = []

# counter for breaks taken by the expert
break_count = 0



A said above, we modeled the front desk and expert operators as resources, calls(called customer in code) and breaks that expert takes are modeled as processes. Breaks taken by the expert makes the expert resource unavailable for 3 mins, so it made sense to model breaks as processes that try to get expert resource.

Class definition for breaks. Breaks are generated for expert operator. When they happen, they take 3 minutes long and expert is not available at that time. 

In [5]:
class Break:
    def __init__(self, env, expert):
        self.env = env
        self.expert = expert
        # it only action is happen, which can be thought as just being initiated
        self.action = env.process(self.happen())
    
    
    def happen(self):
        print(f"Break is on the way \t\t\t\t at {format_time(self.env.now)}")
        with self.expert.request() as req:
            yield req
            print(f"Break started \t\t\t\t\t at {format_time(self.env.now)}")
            yield self.env.timeout(BREAK_DUR)
            print(f"Break ended \t\t\t\t\t at {format_time(self.env.now)}")

* The class definition for the customers arriving at the modeled system. When they are created, they immediatelly initiate a call (i.e. activate the call process). 

* Once a call is initiated, this is registered as a request to the operator resource. The customer is put on hold until the resource activates it back. 

* When the resource is available, the customer is activated and it then initiates first the ask_question_operator process. The duration of a question-answer session is determined randomly according to a LogNormal distribution.

* After this, they are forwarded to expert operator. While waiting for the expert, they may renege after some time taken from an exponential distribution if they aren't addressed by the expert.

* Service time of the expert is taken from another exponential distribution.


In [6]:
class Customer(object):
    def __init__(self, _id, env, operator, expert):
        self.env = env
        self.id = _id
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
    
    
    def call(self):
        print(f"Cust {self.id:4} entered the \t\t FRONT Queue \t at {format_time(self.arrival_t)}")
 
        with operator.request() as op_req :
            yield op_req
            self.wait_front = self.env.now - self.arrival_t
            print(f"Cust {self.id:4} started talking with \t FRONT Desk \t at {format_time(self.env.now)}")
            if self.wait_front:
                    print(f"Cust {self.id:4} waited in \t\t FRONT Queue \t for{format_time(self.wait_front)}")
            first_queue_w_times.append(self.wait_front)
            yield self.env.process(self.ask_question_operator())
            self.finish_operator=self.env.now
            print(f"Cust {self.id:4} entered the \t\t EXPERT Queue \t at {format_time(self.finish_operator)}")
        

        with expert.request() as exp_req: 
            enter_e_q = self.env.now
            # customers may renege in the expert queue
            self.patience=random.expovariate(1/RENEGE_TIME_MEAN)
            results = yield exp_req | env.timeout(self.patience)
            self.wait_exp=self.env.now-self.finish_operator
            second_queue_w_times.append(self.wait_exp) 
            exit_e_q = self.env.now
            global expert_queue_log
            expert_queue_log.append(Expert_Q_Stat(enter_e_q, exit_e_q))

            if exp_req not in results:
                #Leave queue, reneged
                print(f"Cust {self.id:4} exited \t\t SUDDENLY \t at {format_time(self.env.now)}")
                print(f"-- after waiting {format_time(self.wait_exp)}")

            else:
            # We got to the expert
                print(f"Cust {self.id:4} started talking with \t EXPERT \t at {format_time(self.env.now)}")
                if self.wait_exp:
                    print(f"Cust {self.id:4} waited in \t\t EXPERT Queue \t for{format_time(self.wait_exp)}")
                yield self.env.process(self.ask_question_expert())
                self.finish_expert=self.env.now
                print(f"Cust {self.id:4} exited \t\t NORMALLY \t at {format_time(self.finish_expert)}")
        
        global LAST_EXIT
        LAST_EXIT = self.env.now
            
    def ask_question_operator(self):
        service_opr = random.lognormvariate(FIRST_SERVICE_MEAN,FIRST_SERVICE_STD)
        first_service_times.append(service_opr)
        yield self.env.timeout(service_opr)


    def ask_question_expert(self):
        service_exp = random.expovariate(1/EXPERT_SERVICE_MEAN)
        second_service_times.append(service_exp)
        yield self.env.timeout(service_exp)

Generator functions for customers and breaks. 

* Interrarrival times for both are taken from different exponential distributions. 

* Customer generator runs as many times as the number of total customers we want in our simulation, break generator runs through the whole shift.


In [7]:
def customer_generator(env, operator,expert,callnum):
    for i in range(callnum):
        global LAST_CUSTOMER
        LAST_CUSTOMER = LAST_CUSTOMER + 1
        #if env.now >= 60*8:
        #    break
        # lamda is how many for any minute(rate). 1/lamda is how many min for 1 event
        time_between_next_customer = random.expovariate(1/INTERARRIVAL_MEAN)
        customer = Customer(i+1, env, operator,expert)
        #if env.now + time_between_next_customer >= 60*8:
        #    break
        yield env.timeout(time_between_next_customer)
        
def break_generator(env, expert):
    while LAST_CUSTOMER<=CUSTOMER_NUM:
        time_to_break = random.expovariate(1/60)
        #if env.now + time_to_break >= 60*8:
        #    break
        yield env.timeout(time_to_break)
        Break(env, expert)
        global break_count
        break_count += 1

Here, we define our processes and resources and start running the simulation environment!

# To run the simulation for 1000 customers, make sure that CUSTOMER_NUM equals CALLNUM1

In [8]:
env = simpy.Environment()
operator = simpy.Resource(env, capacity = 1)
expert=simpy.Resource(env, capacity = 1)
env.process(customer_generator(env, operator,expert,CUSTOMER_NUM))
env.process(break_generator(env, expert))
end_simulation = simpy.Event(env)
env.run() 

Cust    1 entered the 		 FRONT Queue 	 at  00:00:00:00
Cust    1 started talking with 	 FRONT Desk 	 at  00:00:00:00
Cust    2 entered the 		 FRONT Queue 	 at  00:00:01:05
Cust    3 entered the 		 FRONT Queue 	 at  00:00:02:33
Cust    1 entered the 		 EXPERT Queue 	 at  00:00:05:54
Cust    2 started talking with 	 FRONT Desk 	 at  00:00:05:54
Cust    2 waited in 		 FRONT Queue 	 for 00:00:04:49
Cust    1 started talking with 	 EXPERT 	 at  00:00:05:54
Cust    1 exited 		 NORMALLY 	 at  00:00:13:37
Cust    4 entered the 		 FRONT Queue 	 at  00:00:14:08
Break is on the way 				 at  00:00:14:35
Break started 					 at  00:00:14:35
Cust    2 entered the 		 EXPERT Queue 	 at  00:00:16:23
Cust    3 started talking with 	 FRONT Desk 	 at  00:00:16:23
Cust    3 waited in 		 FRONT Queue 	 for 00:00:13:50
Break ended 					 at  00:00:17:35
Cust    2 started talking with 	 EXPERT 	 at  00:00:17:35
Cust    2 waited in 		 EXPERT Queue 	 for 00:00:01:11
Cust    5 entered the 		 FRONT Queue 	 at  00:00

-- after waiting  00:00:25:36
Cust  190 exited 		 NORMALLY 	 at  01:18:23:10
Break started 					 at  01:18:23:10
Break ended 					 at  01:18:26:10
Cust  191 started talking with 	 EXPERT 	 at  01:18:26:10
Cust  191 waited in 		 EXPERT Queue 	 for 00:00:38:54
Cust  193 entered the 		 FRONT Queue 	 at  01:18:29:20
Cust  193 started talking with 	 FRONT Desk 	 at  01:18:29:20
Cust  191 exited 		 NORMALLY 	 at  01:18:31:42
Cust  193 entered the 		 EXPERT Queue 	 at  01:18:36:52
Cust  193 started talking with 	 EXPERT 	 at  01:18:36:52
Cust  193 exited 		 NORMALLY 	 at  01:18:44:53
Cust  194 entered the 		 FRONT Queue 	 at  01:18:56:14
Cust  194 started talking with 	 FRONT Desk 	 at  01:18:56:14
Cust  194 entered the 		 EXPERT Queue 	 at  01:19:01:21
Cust  194 started talking with 	 EXPERT 	 at  01:19:01:21
Cust  195 entered the 		 FRONT Queue 	 at  01:19:09:45
Cust  195 started talking with 	 FRONT Desk 	 at  01:19:09:45
Cust  194 exited 		 NORMALLY 	 at  01:19:15:33
Cust  195 entered the

Cust  411 started talking with 	 FRONT Desk 	 at  03:23:17:01
Cust  411 entered the 		 EXPERT Queue 	 at  03:23:24:02
Cust  410 exited 		 NORMALLY 	 at  03:23:25:48
Cust  411 started talking with 	 EXPERT 	 at  03:23:25:48
Cust  411 waited in 		 EXPERT Queue 	 for 00:00:01:46
Cust  412 entered the 		 FRONT Queue 	 at  03:23:26:42
Cust  412 started talking with 	 FRONT Desk 	 at  03:23:26:42
Cust  411 exited 		 NORMALLY 	 at  03:23:27:04
Break is on the way 				 at  03:23:27:24
Break started 					 at  03:23:27:24
Cust  413 entered the 		 FRONT Queue 	 at  03:23:27:55
Break ended 					 at  03:23:30:24
Cust  412 entered the 		 EXPERT Queue 	 at  03:23:31:36
Cust  413 started talking with 	 FRONT Desk 	 at  03:23:31:36
Cust  413 waited in 		 FRONT Queue 	 for 00:00:03:41
Cust  412 started talking with 	 EXPERT 	 at  03:23:31:36
Cust  412 exited 		 NORMALLY 	 at  03:23:35:21
Cust  414 entered the 		 FRONT Queue 	 at  03:23:37:27
Cust  413 entered the 		 EXPERT Queue 	 at  03:23:39:37
Cust  

Cust  646 entered the 		 FRONT Queue 	 at  06:09:54:33
Cust  645 entered the 		 EXPERT Queue 	 at  06:09:56:10
Cust  646 started talking with 	 FRONT Desk 	 at  06:09:56:10
Cust  646 waited in 		 FRONT Queue 	 for 00:00:01:36
Cust  644 exited 		 NORMALLY 	 at  06:09:59:34
Cust  645 started talking with 	 EXPERT 	 at  06:09:59:34
Cust  645 waited in 		 EXPERT Queue 	 for 00:00:03:24
Cust  645 exited 		 NORMALLY 	 at  06:09:59:57
Cust  646 entered the 		 EXPERT Queue 	 at  06:10:04:59
Cust  646 started talking with 	 EXPERT 	 at  06:10:04:59
Cust  647 entered the 		 FRONT Queue 	 at  06:10:12:01
Cust  647 started talking with 	 FRONT Desk 	 at  06:10:12:01
Cust  647 entered the 		 EXPERT Queue 	 at  06:10:16:22
Cust  646 exited 		 NORMALLY 	 at  06:10:23:27
Cust  647 started talking with 	 EXPERT 	 at  06:10:23:27
Cust  647 waited in 		 EXPERT Queue 	 for 00:00:07:05
Cust  647 exited 		 NORMALLY 	 at  06:10:28:44
Cust  648 entered the 		 FRONT Queue 	 at  06:10:43:58
Cust  648 started ta

Cust  785 entered the 		 EXPERT Queue 	 at  07:19:10:25
Cust  786 started talking with 	 FRONT Desk 	 at  07:19:10:25
Cust  786 waited in 		 FRONT Queue 	 for 00:00:08:33
Cust  781 exited 		 NORMALLY 	 at  07:19:12:42
Cust  783 started talking with 	 EXPERT 	 at  07:19:12:42
Cust  783 waited in 		 EXPERT Queue 	 for 00:00:17:56
Cust  786 entered the 		 EXPERT Queue 	 at  07:19:17:16
Cust  787 started talking with 	 FRONT Desk 	 at  07:19:17:16
Cust  787 waited in 		 FRONT Queue 	 for 00:00:11:12
Cust  783 exited 		 NORMALLY 	 at  07:19:20:12
Cust  785 started talking with 	 EXPERT 	 at  07:19:20:12
Cust  785 waited in 		 EXPERT Queue 	 for 00:00:09:46
Cust  787 entered the 		 EXPERT Queue 	 at  07:19:21:37
Cust  785 exited 		 NORMALLY 	 at  07:19:21:54
Cust  786 started talking with 	 EXPERT 	 at  07:19:21:54
Cust  786 waited in 		 EXPERT Queue 	 for 00:00:04:37
Cust  786 exited 		 NORMALLY 	 at  07:19:25:33
Cust  787 started talking with 	 EXPERT 	 at  07:19:25:33
Cust  787 waited in 

Break is on the way 				 at  09:08:43:28
Cust  963 exited 		 NORMALLY 	 at  09:08:46:48
Cust  964 started talking with 	 EXPERT 	 at  09:08:46:48
Cust  964 waited in 		 EXPERT Queue 	 for 00:00:17:49
Cust  966 entered the 		 EXPERT Queue 	 at  09:08:47:45
Cust  967 entered the 		 FRONT Queue 	 at  09:08:48:36
Cust  967 started talking with 	 FRONT Desk 	 at  09:08:48:36
Cust  968 entered the 		 FRONT Queue 	 at  09:08:49:37
Cust  967 entered the 		 EXPERT Queue 	 at  09:08:55:27
Cust  968 started talking with 	 FRONT Desk 	 at  09:08:55:27
Cust  968 waited in 		 FRONT Queue 	 for 00:00:05:49
Cust  969 entered the 		 FRONT Queue 	 at  09:08:57:15
Cust  968 entered the 		 EXPERT Queue 	 at  09:09:01:29
Cust  969 started talking with 	 FRONT Desk 	 at  09:09:01:29
Cust  969 waited in 		 FRONT Queue 	 for 00:00:04:13
Cust  966 exited 		 SUDDENLY 	 at  09:09:04:28
-- after waiting  00:00:16:42
Cust  964 exited 		 NORMALLY 	 at  09:09:04:56
Cust  965 started talking with 	 EXPERT 	 at  09:09

Lastly, we print our statistics we gathered from the simulation

In [9]:
util_front = sum(first_service_times) / LAST_EXIT
util_exp = sum(second_service_times) / LAST_EXIT

total_waits = [f_w + e_w for f_w, e_w in itertools.zip_longest(
    first_queue_w_times, 
    second_queue_w_times, 
    fillvalue=0
)]

total_services = [f_s + e_s for f_s, e_s in itertools.zip_longest(
    first_service_times,
    second_service_times,
    fillvalue=0
)]

total_system = [w + s for w, s in itertools.zip_longest(
    total_waits,
    total_services,
    fillvalue=0
)]

# this makes sure everything went smoothly 
assert(len(total_waits) == len(total_system))

max_ratio = max((w/sys if sys else 0 for w, sys in zip(total_waits, total_system)))

# a helper function to calculate avg length of expert queue
def log_to_avg_len(log):
   ins = sorted((entry.enter_e_q for entry in log))
   outs = sorted((entry.exit_e_q for entry in log))
   significant_points = np.array(sorted(ins + outs))
   in_mask = np.array([i in ins for i in significant_points])
   out_mask = np.array([o in outs for o in significant_points])
   queue_len = np.zeros(len(significant_points))

   
   for i, is_in in enumerate(in_mask):
      if is_in:
         queue_len[i:] += 1

   for i, is_in in enumerate(out_mask):
      if is_in:
         queue_len[i:] -= 1

   timedeltas = np.copy(significant_points)
   for i, elem in enumerate(timedeltas):
      timedeltas[i+1:] -= elem

   return np.sum( timedeltas*queue_len ) / outs[-1]

stats = {
    "break_count": break_count,
    "util_front": util_front,
    "util_exp": util_exp,
    "avg_total_wait": sum(total_waits) / len(total_waits),
    "max_wait_2_sys_ratio": max_ratio,
    "avg_len_of_exp_q": log_to_avg_len(expert_queue_log),
}

print(f"Number of breaks taken: {stats['break_count']}")
print(f"Utilization of front desk operator: {stats['util_front']:.2f}")
print(f"Utilization of expert operator: {stats['util_exp']:.2f}")
print(f"Average total waiting time: {format_time(stats['avg_total_wait'])} dd:hh:mm:ss")
print(f"Maximum of total waiting time to system time ratios: {stats['max_wait_2_sys_ratio']:.2f}")
print(f"Average length of expert waiting queue: {stats['avg_len_of_exp_q']:.2f} people.")


Number of breaks taken: 276
Utilization of front desk operator: 0.52
Utilization of expert operator: 0.64
Average total waiting time:  00:00:13:55 dd:hh:mm:ss
Maximum of total waiting time to system time ratios: 0.93
Average length of expert waiting queue: 0.82 people.


In [10]:
print (len(first_queue_w_times))

1000


In [11]:
print (len(second_queue_w_times))

1000


In [12]:
print(format_time(env.now))
print(format_time(LAST_EXIT))

 09:17:03:29
 09:16:38:14
